# 03 · Meridian Bayesian MMM

**Goal:** Train a Meridian model with adstock & saturation; produce a model summary (HTML) and a media summary table.

In [ ]:
# --- install ---
!pip -q install lightweight_mmm "jax[cpu]" arviz matplotlib pandas

# --- imports ---
import pandas as pd
import numpy as np
import arviz as az
import matplotlib.pyplot as plt
from lightweight_mmm import lightweight_mmm as lmmm
from lightweight_mmm import preprocessing, optimize_media

# --- data ---
df = pd.read_csv("marketing_data_clean.csv")  # upload your ../data/marketing_data_clean.csv
y = df["revenue"].values.astype(float)
channels = ["tv_spend","search_spend","social_spend","display_spend"]
X = df[channels].values.astype(float)

# optional controls (price_index, promo, holiday) -> scaled
control_cols = ["price_index","promo","holiday"]
Z = df[control_cols].values.astype(float)

# --- scale & prepare ---
scaler = preprocessing.CustomScaler()
X_scaled = scaler.fit_transform(X)
Z_scaled = preprocessing.CustomScaler().fit_transform(Z)

# --- model ---
mmm = lmmm.LightweightMMM(
    model_name="carryover",
    number_warmup=1000, number_samples=1000, seed=42,
    ad_effect_retention_rate=None,  # let model learn carryover
    weekday_seasonality=False
)

mmm.fit(
    media=X_scaled,
    target=y,
    extra_features=Z_scaled,           # controls
    media_names=channels,
    extra_features_names=control_cols,
)

# --- channel posteriors & contributions ---
contrib = mmm.get_posterior_predictive_contributions().mean(axis=0)  # (time, channels)
roi = mmm.get_roi()  # ROI per channel (posterior mean)

print("ROI by channel:")
for c, r in zip(channels, roi):
    print(f"{c:15s}: {r:.4f}")

# --- plots ---
mmm.plot_media_channel_posteriors()     # coefficients posteriors
plt.show()
mmm.plot_prior_and_posterior()          # model-level
plt.show()

# --- budget optimization (what-if) ---
budget = X_scaled.sum(axis=0) * 1.0     # current total scaled spend
opt = optimize_media.OptimizeMedia(
    media_mix_model=mmm,
    media= X_scaled,
    extra_features=Z_scaled,
    budget= budget * 1.05,              # +5% total budget
    bounds=(X_scaled.min(axis=0), X_scaled.max(axis=0))
)
best, _ = opt.optimize()
best_unscaled = scaler.inverse_transform(best)
pd.DataFrame({"channel":channels, "optimized_spend":best_unscaled}).round(2)
